<a href="https://colab.research.google.com/github/enricsanz-dxc/sesdpw2g4/blob/main/Quantity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SE SDP Wave 2 . Group 4. Use Case

## Quantity prediction
### Time Series
You need to predict two metrics: ALLOCATION and CAPACITY (allocation = ordered quantity, capacity = consumed quantity)  
* An appropriate algorithm is the so called “Holt-Winters-Algorithm”  
* Use TYPE = 1 only, TYPE=2 is not relevant for this time series approach! “1” stands for “households”  
* All data are historical data reaching from 01 Jan 2017 – 02 June 2020. Data reaching from 03 June 2020 to 31 Dec 2020 are simply a guess taken by the NRG-4-U business department and must not be considered within your analysis
* As the business department is under high time pressure, they asked you to predict on the training set only (no test set required)
* Pls consider: if a station does not come with complete historical data it must be removed from the prediction process (this must be documented for the business department)
* Prediction period: 03 June 2020 - 31.12.2021
* Pls remember you need to predict 365 days for every station (guess you have 10 stations, then your result table will be containing 3650 records)





#### Read csv files
* Files has been stored on GITHUB to facilitate location from Notebook.
---
See files on: https://github.com/enricsanz-dxc/sesdpw2g4


In [1]:
import pandas as pd
import io
import datetime


In [2]:
# Import  files from GITHUB
url_inputdata_timeseries = 'https://github.com/enricsanz-dxc/sesdpw2g4/blob/main/files/inputdata_timeseries.csv?raw=true'

df_it = pd.read_csv(url_inputdata_timeseries, index_col=3, parse_dates=True)

In [3]:
# Filter TYPE 1 
df_it=df_it[df_it['TYPE']==1]

In [4]:
df_it['STATION'].value_counts()

L1001121    1553
L2000260    1553
L2000360    1553
L1000896    1553
L2000746    1553
L1001105    1553
L1000920    1553
L1000488    1553
L2000932    1553
L2000361    1553
L1000923    1553
L2000022    1553
L1000947    1553
L1000601    1553
L1001138    1553
L1000899    1553
L1000927    1553
L2000907    1553
L2000703    1553
L2000600    1553
L1001052    1553
L2001083    1553
L1000672    1553
L1000945    1553
L2001082    1553
L1001094    1553
L1001118    1553
L1001103    1553
L1001076    1553
L1000732    1553
L1000915    1553
L2000943    1553
L1001117    1553
L1001123    1553
L1000780    1553
L1001107    1553
L1000950    1553
L2000857    1553
L1000970    1553
L2000800    1553
L1000955    1553
Name: STATION, dtype: int64

In [5]:
# Filter Stations with complete timeseries 
df_it=df_it.groupby('STATION').filter(lambda x: len(x) >= 1553)

In [6]:
df_it.head()


,TYPE,STATION,CALENDARYEAR,ALLOCATION,CAPACITY
DAY,,,,,
2017-10-01,1,L1000488,2017,26580795.0,27445866.0
2017-10-02,1,L1000488,2017,22755336.0,27445866.0
2017-10-03,1,L1000488,2017,25024228.0,27445866.0
2017-10-04,1,L1000488,2017,25024228.0,27445866.0
2017-10-05,1,L1000488,2017,25265545.0,27445866.0


In [7]:
df_it.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 63673 entries, 2017-10-01 to 2021-12-31
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TYPE          63673 non-null  int64  
 1   STATION       63673 non-null  object 
 2   CALENDARYEAR  63673 non-null  int64  
 3   ALLOCATION    63673 non-null  float64
 4   CAPACITY      63673 non-null  float64
dtypes: float64(2), int64(2), object(1)
memory usage: 2.9+ MB


In [8]:
# Add columns with year, month, and weekday name
df_it['Year'] = df_it.index.year
df_it['Month'] = df_it.index.month
# Display a random sampling of 5 rows
df_it.sample(5, random_state=0)

,TYPE,STATION,CALENDARYEAR,ALLOCATION,CAPACITY,Year,Month
DAY,,,,,,,
2021-01-24,1,L2000932,2021,0.0,0.0,2021,1
2020-07-11,1,L2000943,2020,0.0,868407.0,2020,7
2021-01-08,1,L2000703,2021,0.0,1.0,2021,1
2018-09-22,1,L1001118,2018,1070.0,1392.0,2018,9
2018-02-12,1,L1000488,2018,25115844.0,25324021.0,2018,2


In [9]:
#Filter only data from  01 Jan 2017 – 02 June 2020
df_f=df_it.loc['2017-01-01':'2020-06-02']

In [10]:
df_f.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 40016 entries, 2017-10-01 to 2020-06-02
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TYPE          40016 non-null  int64  
 1   STATION       40016 non-null  object 
 2   CALENDARYEAR  40016 non-null  int64  
 3   ALLOCATION    40016 non-null  float64
 4   CAPACITY      40016 non-null  float64
 5   Year          40016 non-null  int64  
 6   Month         40016 non-null  int64  
dtypes: float64(2), int64(4), object(1)
memory usage: 2.4+ MB


In [11]:
df_f['STATION'].value_counts()

L1001121    976
L1001117    976
L1001123    976
L1000780    976
L1001107    976
L1000950    976
L2000857    976
L1000970    976
L1000899    976
L1000927    976
L2000800    976
L1000672    976
L2000907    976
L1001052    976
L2000260    976
L2000600    976
L2000360    976
L2000703    976
L1000955    976
L1001138    976
L2000943    976
L2001083    976
L1000601    976
L1000945    976
L2000746    976
L2001082    976
L1001094    976
L1001105    976
L1001118    976
L1001103    976
L1001076    976
L1000920    976
L1000488    976
L2000932    976
L1000732    976
L2000361    976
L1000923    976
L2000022    976
L1000915    976
L1000947    976
L1000896    976
Name: STATION, dtype: int64

In [12]:
import matplotlib.pyplot as plt

In [13]:
import seaborn as sns
# Use seaborn style defaults and set the default figure size
sns.set(rc={'figure.figsize':(11, 4)})


In [14]:
df_it.index

DatetimeIndex(['2017-10-01', '2017-10-02', '2017-10-03', '2017-10-04',
               '2017-10-05', '2017-10-06', '2017-10-07', '2017-10-08',
               '2017-10-09', '2017-10-10',
               ...
               '2021-12-22', '2021-12-23', '2021-12-24', '2021-12-25',
               '2021-12-26', '2021-12-27', '2021-12-28', '2021-12-29',
               '2021-12-30', '2021-12-31'],
              dtype='datetime64[ns]', name='DAY', length=63673, freq=None)

In [15]:
df_it=df_it.set_index('DAY')

KeyError: ignored

In [ ]:
df_it.index

In [ ]:
df_it.plot(x="DAY",y=["ALLOCATION", "CAPACITY"], figsize=(15,4))
df_it.plot(x="DAY", y=["CAPACITY", "ALLOCATION"], style='.')

In [ ]:
df_it['ALLOCATION'].plot(linewidth=0.5);

## Prediction for the IT department
You need to predict the CAPA_QUANTITY only
* This time use POINTTYPE=2 only (“2” stands for industry customers)
* Pls create a training set reaching from 01 Jan 2016 – 31 Dec 2018 and test set reaching from 01 Jan 2019 – 31 Dec 2019
* As prediction algorithms you should be considering multivariate linear and non-linear regression and Support Vector Machines (SVM). All these algorithms should be part of the ML tools. Otherwise, take an algorithm out of your choice.
* You need to find the best combination (= combination with best prediction quality) for all possible input factor combinations
* Pls consider that the input factors themselves need to be predicted when predicting the result for 2021.
* Prediction period: 03 June 2020 - 31.12.2021
* Pls remember you need to predict 365 days for every station (guess you have 10 stations, then your result table will be containing 3650 records)

In [ ]:
url_quantityPrediction_quantities = 'https://github.com/enricsanz-dxc/sesdpw2g4/blob/main/files/quantityPrediction_quantities.csv?raw=true'
url_quantityPrediction_stockPrices = 'https://github.com/enricsanz-dxc/sesdpw2g4/raw/main/files/quantityPrediction_stockPrices.csv?raw=true'
url_quantityPrediction_temperatures = 'https://github.com/enricsanz-dxc/sesdpw2g4/raw/main/files/quantityPrediction_temperatures.csv?raw=true'


df_qp_q = pd.read_csv(url_quantityPrediction_quantities,parse_dates=['stockdate'])
df_qd_sp = pd.read_csv(url_quantityPrediction_stockPrices,parse_dates=['BUSINESSDATE'])
df_qs_t = pd.read_csv(url_quantityPrediction_temperatures,parse_dates=['tempdate'])



In [18]:
df_qd_sp.head()


,stockdate,ncg,powerpool,ttf,ztp,powerpool_uk_ocm
0,20160102,1971,2077,2075,20801,2077
1,20160103,2144,2158,2101,20802,2158
2,20160104,2155,2153,21157,21266,2153
3,20160105,2142,2169,21231,21441,2169
4,20160106,2198,2181,21412,21591,2181


In [19]:
df_qp_q.head()

,STATIONS,PTYPE,BUSINESSDATE,CAPA_QUANTITY
0,L2000800,2,20180422,94001640
1,L2000800,2,20180503,74321640
2,L2000800,2,20180504,74321640
3,L2000800,2,20180505,62321640
4,L2000800,2,20180524,119153640


In [20]:
df_qs_t.head()

,tempdate,temp_de,temp_it,temp_at,temp_pl,temp_ru,temp_ukr
0,20160102,8.2,-2.1,12.2,7.0,2.6,-1.5
1,20160103,6.1,-2.4,12.7,5.9,6.6,2.6
2,20160104,5.5,-0.3,11.0,6.4,5.9,2.5
3,20160105,4.9,1.8,10.8,3.2,6.2,3.1
4,20160106,3.3,2.0,8.4,2.4,4.7,3.4


Filter POINTTYPE=2

In [22]:
# Filter TYPE 1 
df_qp_q=df_qp_q[df_qp_q['PTYPE']==2]

In [24]:
df_qp_q.head()

,STATIONS,PTYPE,BUSINESSDATE,CAPA_QUANTITY
0,L2000800,2,20180422,94001640
1,L2000800,2,20180503,74321640
2,L2000800,2,20180504,74321640
3,L2000800,2,20180505,62321640
4,L2000800,2,20180524,119153640


Create a training set reaching from 01 Jan 2016 – 31 Dec 2018 and test set reaching from 01 Jan 2019 – 31 Dec 2019